### Import library

In [1]:
import requests
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import sys
!{sys.executable} -m pip install folium
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize


from sklearn.cluster import KMeans

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[0]

    100% |████████████████████████████████| 92kB 7.6MB/s eta 0:00:01
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: MarkupSafe>=0.23 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from jinja2->folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/

#### Clean data

In [2]:
df.rename(columns={0:'Postalcode',1:'Borough',2:'Neighborhood'},inplace=True)
df = df.iloc[1:]
df = df[df.Borough != 'Not assigned']
for i in range(0,df.shape[0]):
    if df.iloc[i][2] == 'Not assigned':
        df.iloc[i][2] = df.iloc[i][1]
        i = i+1
df = df.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

#### Shape of dataframe

In [3]:
print(df.shape)
df.head()

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Import data from geospatial

In [4]:
geo=pd.read_csv('http://cocl.us/Geospatial_data')
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merge data

In [5]:
df2=pd.concat([df,geo],axis=1)
df2.drop(df2.columns[3],axis=1,inplace=True)

In [6]:
df2.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Get latitude and longitude of Toronto

In [8]:
address = 'Toronto,Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_y = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude_x, longitude_y))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [9]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude_x, longitude_y], zoom_start=10)

# add markers to map
for lat, lng, bor, nei in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    
    label = '{}, {}'.format(nei, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [10]:
Scarborough_data = df2[df2['Borough'] == 'Scarborough'].reset_index(drop=True)
Scarborough_data

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


#### Geographical coordinates of Scarborough

In [11]:
address = 'Scarborough,Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_s = location.latitude
longitude_s = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_s, longitude_s))
/u

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


NameError: name 'u' is not defined

### Identify neighborhoods in Scarborough

In [13]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude_s, longitude_s], zoom_start=10)

# add markers to map
for lat, lng, bor, nei in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'],Scarborough_data['Borough'], Scarborough_data['Neighborhood']):
    
    label = '{}, {}'.format(nei, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

#### Foursquare Credentials and Version

In [14]:
CLIENT_ID = 'P5YYMBNDOKLUPGTVAMN02B24QMVGNTNL1G3EH5PBCTPSMYS2' # your Foursquare ID
CLIENT_SECRET = 'PXB0XEEDZAEI3AMGCZGXFSW4SSFY2A1YHESACTP0UEMQKXIC' # your Foursquare Secret
VERSION = '20190326' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: P5YYMBNDOKLUPGTVAMN02B24QMVGNTNL1G3EH5PBCTPSMYS2
CLIENT_SECRET:PXB0XEEDZAEI3AMGCZGXFSW4SSFY2A1YHESACTP0UEMQKXIC


#### Exploring the first neighborhood of Scarborough

In [15]:
first_nei=Scarborough_data['Neighborhood'][0]
first_nei

'Rouge, Malvern'

In [16]:
first_nei_lat=Scarborough_data.loc[0,'Latitude']
first_nei_lon=Scarborough_data.loc[0,'Longitude']
print('Latitude and longitude values of {} are {}, {}.'.format(first_nei, 
                                                               first_nei_lat, 
                                                               first_nei_lon))

Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


Now, let's get the top 100 venues that are in Malvern within a radius of 500 meters.

In [17]:

radius = 500 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    first_nei_lat, 
    first_nei_lon, 
    radius, 
    LIMIT)

In [18]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c9b89ffdd579725dcc4a142'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bb6b9446edc76b0d771311c-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16e941735',
         'name': 'Fast Food Restaurant',
         'pluralName': 'Fast Food Restaurants',
         'primary': True,
         'shortName': 'Fast Food'}],
       'id': '4bb6b9446edc76b0d771311c',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Morningside & Sheppard',
        'distance': 387,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80744841934756,
          'ln

#### Since we havent received many results in this neighborhood let's select a different neighborhood

In [19]:
second_nei_name=Scarborough_data.loc[15,'Neighborhood']
second_nei_lat=Scarborough_data.loc[15,'Latitude']
second_nei_lon=Scarborough_data.loc[15,'Longitude']
print('Latitude and longitude values of {} are {}, {}.'.format(second_nei_name, 
                                                               second_nei_lat, 
                                                               second_nei_lon))

Latitude and longitude values of L'Amoreaux West, Steeles West are 43.799525200000005, -79.3183887.


In [20]:
radius = 500 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    second_nei_lat, 
    second_nei_lon, 
    radius, 
    LIMIT)

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c9b8a231ed21927283c04f4'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4df28ea0e4cda09e6da0129a-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d145941735',
         'name': 'Chinese Restaurant',
         'pluralName': 'Chinese Restaurants',
         'primary': True,
         'shortName': 'Chinese'}],
       'id': '4df28ea0e4cda09e6da0129a',
       'location': {'address': '2900 Warden Ave',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'at Finch Ave. E',
        'distance': 72,
        'formattedAddress': ['2900 Warden Ave (at Finch Ave. E)',
         'Toronto ON M1W 2S8',
         'Canada'],
        'labe

In [22]:
results['response']['groups'][0]['items'][0]['venue']['categories'][0]['name']

'Chinese Restaurant'

#### All the information is in the items key. Lets create a get_category_type function

In [23]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.columns

Index(['reasons.count', 'reasons.items', 'referralId', 'venue.categories',
       'venue.id', 'venue.location.address', 'venue.location.cc',
       'venue.location.city', 'venue.location.country',
       'venue.location.crossStreet', 'venue.location.distance',
       'venue.location.formattedAddress', 'venue.location.labeledLatLngs',
       'venue.location.lat', 'venue.location.lng',
       'venue.location.neighborhood', 'venue.location.postalCode',
       'venue.location.state', 'venue.name', 'venue.photos.count',
       'venue.photos.groups'],
      dtype='object')

In [24]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Mr Congee Chinese Cuisine 龍粥記,"[{'primary': True, 'id': '4bf58dd8d48988d14594...",43.798879,-79.318335
1,Phoenix Restaurant 金鳳餐廳,"[{'primary': True, 'id': '4bf58dd8d48988d14594...",43.798198,-79.318432
2,Subway,"[{'primary': True, 'id': '4bf58dd8d48988d1c594...",43.798983,-79.318838
3,Price Chopper,"[{'primary': True, 'id': '4bf58dd8d48988d11895...",43.799445,-79.318563
4,Tim Hortons,"[{'primary': True, 'id': '4bf58dd8d48988d1e093...",43.798281,-79.318317
5,Shoppers Drug Mart,"[{'primary': True, 'id': '4bf58dd8d48988d10f95...",43.799670,-79.319315
6,KFC,"[{'primary': True, 'id': '4bf58dd8d48988d16e94...",43.798938,-79.318854
7,Eggsmart,"[{'primary': True, 'id': '4bf58dd8d48988d14394...",43.796375,-79.318681
8,McDonald's,"[{'primary': True, 'id': '4bf58dd8d48988d16e94...",43.798880,-79.318724
9,Pizza Pizza,"[{'primary': True, 'id': '4bf58dd8d48988d1ca94...",43.797909,-79.318113


In [26]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Mr Congee Chinese Cuisine 龍粥記,Chinese Restaurant,43.798879,-79.318335
1,Phoenix Restaurant 金鳳餐廳,Chinese Restaurant,43.798198,-79.318432
2,Subway,Sandwich Place,43.798983,-79.318838
3,Price Chopper,Grocery Store,43.799445,-79.318563
4,Tim Hortons,Coffee Shop,43.798281,-79.318317
5,Shoppers Drug Mart,Pharmacy,43.799670,-79.319315
6,KFC,Fast Food Restaurant,43.798938,-79.318854
7,Eggsmart,Breakfast Spot,43.796375,-79.318681
8,McDonald's,Fast Food Restaurant,43.798880,-79.318724
9,Pizza Pizza,Pizza Place,43.797909,-79.318113


#### Explore Neighborhoods in Scarborough

#### Let's create a function to repeat the same process to all the neighborhoods in Scarborough

In [27]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Following script run the above function on each neighborhood and create a new dataframe called scarborough_venues.

In [28]:
scarborough_venues = getNearbyVenues(names=Scarborough_data['Neighborhood'],
                                   latitudes=Scarborough_data['Latitude'],
                                   longitudes=Scarborough_data['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge


In [29]:
print(scarborough_venues.shape)
scarborough_venues.head()

(85, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


#### Venues per neighborhood

In [30]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clairlea, Golden Mile, Oakridge",9,9,9,9,9,9
"Clarks Corners, Sullivan, Tam O'Shanter",9,9,9,9,9,9
"Cliffcrest, Cliffside, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",6,6,6,6,6,6
"East Birchmount Park, Ionview, Kennedy Park",6,6,6,6,6,6


#### Unique Categories

In [31]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 52 uniques categories.


### Analyze Each Neighborhood

In [32]:
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]
scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,...,Print Shop,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
scarborough_onehot.shape

(85, 53)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [34]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,...,Print Shop,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.000000,0.000,0.0,0.200000,0.000000,0.000000,0.00,...,0.0,0.000,0.200000,0.00,0.00,0.000000,0.000,0.000000,0.000000,0.000000
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.00,...,0.0,0.000,0.000000,0.00,0.00,0.000000,0.000,0.000000,0.000000,0.000000
2,"Birch Cliff, Cliffside West",0.0,0.000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.25,...,0.0,0.000,0.000000,0.25,0.00,0.000000,0.000,0.000000,0.000000,0.000000
3,Cedarbrae,0.0,0.125,0.125000,0.125,0.0,0.000000,0.000000,0.000000,0.00,...,0.0,0.000,0.000000,0.00,0.00,0.000000,0.000,0.125000,0.000000,0.000000
4,"Clairlea, Golden Mile, Oakridge",0.0,0.000,0.222222,0.000,0.0,0.000000,0.222222,0.111111,0.00,...,0.0,0.000,0.000000,0.00,0.00,0.111111,0.000,0.000000,0.000000,0.000000
5,"Clarks Corners, Sullivan, Tam O'Shanter",0.0,0.000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.00,...,0.0,0.000,0.000000,0.00,0.00,0.000000,0.000,0.111111,0.000000,0.000000
6,"Cliffcrest, Cliffside, Scarborough Village West",0.5,0.000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.00,...,0.0,0.000,0.000000,0.00,0.00,0.000000,0.000,0.000000,0.000000,0.000000
7,"Dorset Park, Scarborough Town Centre, Wexford ...",0.0,0.000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.00,...,0.0,0.000,0.000000,0.00,0.00,0.000000,0.000,0.000000,0.000000,0.166667
8,"East Birchmount Park, Ionview, Kennedy Park",0.0,0.000,0.000000,0.000,0.0,0.000000,0.000000,0.166667,0.00,...,0.0,0.000,0.000000,0.00,0.00,0.000000,0.000,0.000000,0.000000,0.000000
9,"Guildwood, Morningside, West Hill",0.0,0.000,0.000000,0.000,0.0,0.125000,0.000000,0.000000,0.00,...,0.0,0.125,0.000000,0.00,0.00,0.000000,0.125,0.000000,0.000000,0.000000


In [35]:
### Each neighborhood along with the top 5 most common venues

In [36]:

num_top_venues = 5
for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp =scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                venue  freq
0              Lounge   0.2
1      Breakfast Spot   0.2
2      Sandwich Place   0.2
3  Chinese Restaurant   0.2
4      Clothing Store   0.2


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                 venue  freq
0                 Park  0.67
1           Playground  0.33
2  American Restaurant  0.00
3            Pet Store  0.00
4    Korean Restaurant  0.00


----Birch Cliff, Cliffside West----
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2                   Café  0.25
3  General Entertainment  0.25
4               Pharmacy  0.00


----Cedarbrae----
                venue  freq
0              Lounge  0.12
1              Bakery  0.12
2                Bank  0.12
3     Thai Restaurant  0.12
4  Athletics & Sports  0.12


----Clairlea, Golden Mile, Oakridge----
           venue  freq
0         Bakery  0.22
1       Bus Line  0.22
2  Metro Station  0.11
3    Bus Station  0.11
4   

#### Function to sort the venues in descending order.

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Breakfast Spot,Sandwich Place,Lounge,Chinese Restaurant,College Stadium,History Museum,Hakka Restaurant,Grocery Store,General Entertainment
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Vietnamese Restaurant,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,History Museum,Hakka Restaurant,Grocery Store,Fried Chicken Joint
3,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Lounge,Caribbean Restaurant,Fried Chicken Joint,Vietnamese Restaurant,Department Store
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Soccer Field,Bus Station,Fast Food Restaurant,Metro Station,Park,Vietnamese Restaurant,College Stadium,Hakka Restaurant
5,"Clarks Corners, Sullivan, Tam O'Shanter",Pizza Place,Chinese Restaurant,Thai Restaurant,Italian Restaurant,Fried Chicken Joint,Fast Food Restaurant,Pharmacy,Noodle House,Clothing Store,Grocery Store
6,"Cliffcrest, Cliffside, Scarborough Village West",American Restaurant,Motel,Indian Restaurant,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
7,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Latin American Restaurant,Pet Store,Bar,Breakfast Spot,History Museum,Hakka Restaurant,Grocery Store
8,"East Birchmount Park, Ionview, Kennedy Park",Discount Store,Coffee Shop,Hobby Shop,Bus Station,Department Store,College Stadium,History Museum,Hakka Restaurant,Grocery Store,General Entertainment
9,"Guildwood, Morningside, West Hill",Intersection,Spa,Breakfast Spot,Rental Car Location,Electronics Store,Pizza Place,Medical Center,Mexican Restaurant,Coffee Shop,Grocery Store


#### Cluster Neighborhoods

In [39]:
kclusters = 4

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

kmeans.labels_

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 0, 2], dtype=int32)

In [40]:
scarborough_merged = Scarborough_data[0:16]

scarborough_merged['Cluster Labels'] = kmeans.labels_

scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Coffee Shop,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0,History Museum,Bar,Vietnamese Restaurant,College Stadium,Hobby Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Intersection,Spa,Breakfast Spot,Rental Car Location,Electronics Store,Pizza Place,Medical Center,Mexican Restaurant,Coffee Shop,Grocery Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Indian Restaurant,Korean Restaurant,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Lounge,Caribbean Restaurant,Fried Chicken Joint,Vietnamese Restaurant,Department Store


#### Map cluster

In [41]:
map_clusters = folium.Map(location=[latitude_s, longitude_s], zoom_start=11)

x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)

markers_colors = []
for lat, lon, nei , cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#2adddd', '#d4dd80', '#ff0000']


### Examine Clusters

#### Cluster 1

In [42]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Highland Creek, Rouge Hill, Port Union",0,History Museum,Bar,Vietnamese Restaurant,College Stadium,Hobby Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
14,"Agincourt North, L'Amoreaux East, Milliken, St...",0,Park,Playground,Vietnamese Restaurant,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


#### Cluster 2

In [43]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge, Malvern",1,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Coffee Shop,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store
2,"Guildwood, Morningside, West Hill",1,Intersection,Spa,Breakfast Spot,Rental Car Location,Electronics Store,Pizza Place,Medical Center,Mexican Restaurant,Coffee Shop,Grocery Store
3,Woburn,1,Coffee Shop,Indian Restaurant,Korean Restaurant,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,Cedarbrae,1,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Lounge,Caribbean Restaurant,Fried Chicken Joint,Vietnamese Restaurant,Department Store
5,Scarborough Village,1,Playground,Vietnamese Restaurant,Coffee Shop,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
6,"East Birchmount Park, Ionview, Kennedy Park",1,Discount Store,Coffee Shop,Hobby Shop,Bus Station,Department Store,College Stadium,History Museum,Hakka Restaurant,Grocery Store,General Entertainment
7,"Clairlea, Golden Mile, Oakridge",1,Bakery,Bus Line,Soccer Field,Bus Station,Fast Food Restaurant,Metro Station,Park,Vietnamese Restaurant,College Stadium,Hakka Restaurant
8,"Cliffcrest, Cliffside, Scarborough Village West",1,American Restaurant,Motel,Indian Restaurant,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
9,"Birch Cliff, Cliffside West",1,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,History Museum,Hakka Restaurant,Grocery Store,Fried Chicken Joint
10,"Dorset Park, Scarborough Town Centre, Wexford ...",1,Indian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Latin American Restaurant,Pet Store,Bar,Breakfast Spot,History Museum,Hakka Restaurant,Grocery Store


#### Cluster 3

In [44]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,"L'Amoreaux West, Steeles West",2,Chinese Restaurant,Fast Food Restaurant,Coffee Shop,Thrift / Vintage Store,Grocery Store,Japanese Restaurant,Breakfast Spot,Sandwich Place,Pizza Place,Pharmacy


#### Cluster 4

In [45]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,"Clarks Corners, Sullivan, Tam O'Shanter",3,Pizza Place,Chinese Restaurant,Thai Restaurant,Italian Restaurant,Fried Chicken Joint,Fast Food Restaurant,Pharmacy,Noodle House,Clothing Store,Grocery Store
